In [1]:
import os
from pathlib import Path

ROOT_PATH = '.'
SRC_PATH = os.path.join(ROOT_PATH, "src")
DATA_PATH = os.path.join(ROOT_PATH, "data")

In [2]:
import sqlite3


def dict_factory(cursor, row):
    fields = [column[0] for column in cursor.description]
    return {key: value for key, value in zip(fields, row)}

DB_PATH = os.path.join(DATA_PATH, "raw", "card_database.sqlite")
con = sqlite3.connect(DB_PATH)
con.row_factory = dict_factory
cur = con.cursor()

row = cur.execute("SELECT name, scryfallId, id FROM cards LIMIT 1;").fetchone()
scryfallId = row['scryfallId']
card_url = f"https://api.scryfall.com/cards/{scryfallId}?format=image"


In [3]:
row = cur.execute("SELECT COUNT(*) FROM cards").fetchone()
print(row)

{'COUNT(*)': 75840}


In [58]:
import shutil
import requests

def download_image(url, dest):
    response = requests.get(url, stream=True)
    with open(dest, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

In [59]:
download_image(card_url, f"{row['id']}.png")

In [73]:
cards[0]

{'name': "Ancestor's Chosen",
 'scryfallId': '7a5cd03c-4227-4551-aa4b-7d119f0468b5',
 'id': 1}

In [71]:
cards = cur.execute("SELECT name, scryfallId, id FROM cards").fetchall()

IMAGES_DESTINATION = os.path.join(DATA_PATH, "raw", "card_images")
if not os.path.exists(IMAGES_DESTINATION): 
  os.makedirs(IMAGES_DESTINATION)


def download_card(card):
    scryfallId = card['scryfallId']
    image_url = f"https://api.scryfall.com/cards/{scryfallId}?format=image&version=normal"
    
    card_filename = f"{card['id']}.png"
    card_path = os.path.join(IMAGES_DESTINATION, card_filename)
    
    if os.path.isfile(card_path):
        print("ALreday exist")
    else:
        download_image(image_url, card_path) 

In [72]:
from joblib import Parallel, delayed

results = Parallel(n_jobs=100)(delayed(download_card)(card) for card  in cards)
print(results)

KeyboardInterrupt: 